In [2]:
# Torch
import torch
import torch.optim as optim
from torcheval.metrics import *

import dill as pickle
from captum.attr import *
import random
import numpy as np

# Custom modules
from preprocessing_post_fastsurfer.subject import *
from preprocessing_post_fastsurfer.vis import *
from ozzy_torch_utils.split_dataset import *
from ozzy_torch_utils.subject_dataset import *
from ozzy_torch_utils.plot import *
from ozzy_torch_utils.train_nn import *
from ozzy_torch_utils.model_parameters import *
from ozzy_torch_utils.init_dataloaders import *
from explain_pointnet import *

In [3]:
# Load model
pickle_pathname = "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/runs/run_18-03-2025_15-35-05/run_18-03-2025_15-35-05_params.pkl"

with open(pickle_pathname, 'rb') as file:
    
    model_parameters = pickle.load(file)
    
model = model_parameters.model

In [4]:
# Load dataset
data_path = "/uolstore/home/users/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/scratch-disk/full-datasets/hcampus-1.5T-cohort"

subject_list = find_subjects_parallel(data_path)

Interesting experiment comparing attributions from two permutations of the same cloud

In [5]:
subject = sample(subject_list, 1)[0]

cloud = np.load(os.path.join(subject.path, "Left-Hippocampus_aligned_cropped_mesh_downsampledcloud.npy"))
    

In [6]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

attributions_orig, pred_research_group_orig = pointnet_ig(model, cloud, device)

indices_shuffled = np.random.permutation(cloud.shape[0])

cloud_shuffled = cloud[indices_shuffled, :]

attributions_shuffle, pred_research_group_shuffle = pointnet_ig(model, cloud_shuffled, device)

attributions_unshuffle = np.zeros_like(attributions_shuffle)

attributions_unshuffle[np.argsort(indices_shuffled), :] = attributions_shuffle

attributions_diff = np.abs(attributions_orig - attributions_unshuffle)

print(attributions_diff)

if pred_research_group_orig != pred_research_group_shuffle:
    
    print("Research groups are different after shuffle")
    

[[4.52784230e-07 1.04532260e-07 6.22016916e-08]
 [9.18408782e-07 4.80062276e-07 9.31519020e-07]
 [1.10254554e-10 1.03085823e-10 4.67121312e-10]
 ...
 [5.84079235e-04 9.14718875e-04 5.08224535e-04]
 [1.89030954e-02 1.29853705e-02 9.51814267e-03]
 [2.26426982e-04 5.28159300e-05 5.64208816e-05]]


In [7]:
vis_attributions(attributions_orig, subject, cloud, pred_research_group_orig)

vis_attributions(attributions_unshuffle, subject, cloud, pred_research_group_shuffle)

vis_attributions(attributions_diff, subject, cloud, pred_research_group_orig)

Widget(value='<iframe src="http://localhost:45247/index.html?ui=P_0x7f19eae70ec0_0&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:45247/index.html?ui=P_0x7f18f42b48f0_1&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:45247/index.html?ui=P_0x7f18f4697b00_2&reconnect=auto" class="pyvi…